# Databricks notebook source
 <div style="text-align: center; line-height: 0; padding-top: 9px;">
   <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
 </div>

 # Fine-tuning LLMs
  Many LLMs are general purpose models trained on a broad range of data and use cases. This enables them to perform well in a variety of applications, as shown in previous modules. It is not uncommon though to find situations where applying a general purpose model performs unacceptably for specific dataset or use case. This often does not mean that the general purpose model is unusable. Perhaps, with some new data and additional training the model could be improved, or fine-tuned, such that it produces acceptable results for the specific use case.
  
  Fine-tuning uses a pre-trained model as a base and continues to train it with a new, task targeted dataset. Conceptually, fine-tuning leverages that which has already been learned by a model and aims to focus its learnings further for a specific task.

  It is important to recognize that fine-tuning is model training. The training process remains a resource intensive, and time consuming effort. Albeit fine-tuning training time is greatly shortened as a result of having started from a pre-trained model. The model training process can be accelerated through the use of tools like Microsoft's [DeepSpeed](https://github.com/microsoft/DeepSpeed).

  This notebook will explore how to perform fine-tuning at scale.

 ### ![Dolly](https://files.training.databricks.com/images/llm/dolly_small.png) Learning Objectives
 1. Prepare a novel dataset
 1. Fine-tune the `t5-small` model to classify movie reviews.
 1. Leverage DeepSpeed to enhance training process.

In [ ]:
assert "gpu" in spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion"), "THIS LAB REQUIRES THAT A GPU MACHINE AND RUNTIME IS UTILIZED."

 ## Classroom Setup
 Later sections of this notebook will leverage the DeepSpeed package. DeepSpeed has some additional dependencies that need to be installed in the Databricks environment. The dependencies vary based upon which MLR runtime is being used. The below commands add the necessary libraries accordingly. It is convenient to perform this step at the start of the Notebook to avoid future restarts of the Python kernel.

In [4]:
 # %sh
 # mkdir -p /tmp/externals/cuda

 # wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcurand-dev-11-7_10.2.10.50-1_amd64.deb -P /tmp/externals/cuda
 # wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcusparse-dev-11-7_11.7.3.50-1_amd64.deb -P /tmp/externals/cuda
 # wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcublas-dev-11-7_11.10.1.25-1_amd64.deb -P /tmp/externals/cuda
 # wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcusolver-dev-11-7_11.4.0.1-1_amd64.deb -P /tmp/externals/cuda

 # dpkg -i /tmp/externals/cuda/libcurand-dev-11-7_10.2.10.50-1_amd64.deb
 # dpkg -i /tmp/externals/cuda/libcusparse-dev-11-7_11.7.3.50-1_amd64.deb
 # dpkg -i /tmp/externals/cuda/libcublas-dev-11-7_11.10.1.25-1_amd64.deb
 # dpkg -i /tmp/externals/cuda/libcusolver-dev-11-7_11.4.0.1-1_amd64.deb

In [ ]:
 %pip install deepspeed==0.9.1 py-cpuinfo==9.0.0

In [ ]:
 %run ../Includes/Classroom-Setup

In [5]:
print(f"Username:          {DA.username}")
print(f"Working Directory: {DA.paths.working_dir}")



 %load_ext autoreload
 %autoreload 2

IndentationError: unexpected indent (25381625.py, line 6)

 Creating a local temporary directory on the Driver. This will serve as a root directory for the intermediate model checkpoints created during the training process. The final model will be persisted to DBFS.

In [1]:
import tempfile

tmpdir = tempfile.TemporaryDirectory()
local_training_root = tmpdir.name

local_training_root

'/tmp/tmpzux8klb4'

 ## Fine-Tuning

In [3]:
import os
import pandas as pd
import transformers as tr
from datasets import load_dataset,load_dataset

 ### Step 1 - Data Preparation

 The first step of the fine-tuning process is to identify a specific task and supporting dataset. In this notebook, we will consider the specific task to be classifying movie reviews. This idea is generally simple task where a movie review is provided as plain-text and we would like to determine whether or not the review was positive or negative.

 The [IMDB dataset](https://huggingface.co/datasets/imdb) can be leveraged as a supporting dataset for this task. The dataset conveniently provides both a training and testing dataset with labeled binary sentiments, as well as a dataset of unlabeled data.

In [4]:
imdb_ds = load_dataset("imdb")

In [21]:
sample_train_ds = imdb_ds['train'].select(range(10))
sample_test_ds = imdb_ds['test'].select(range(10))

In [22]:
imdb_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

 ### Step 2 - Select pre-trained model

 The next step of the fine-tuning process is to select a pre-trained model. We will consider using the [T5](https://huggingface.co/docs/transformers/model_doc/t5) [[paper]](https://arxiv.org/pdf/1910.10683.pdf) family of models for our fine-tuning purposes. The T5 models are text-to-text transformers that have been trained on a multi-task mixture of unsupervised and supervised tasks. They are well suited for tasks such as summarization, translation, text classification, question answering, and more.
 
 * Recall from Module 1, Hugging Face provides the [Auto*](https://huggingface.co/docs/transformers/model_doc/auto) suite of objects to conveniently instantiate the various components associated with a pre-trained model. Here, we use the [AutoTokenizer](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer) to load in the tokenizer that is associated with the `t5-small` model.
 The `t5-small` version of the T5 models has 60 million parameters. This slimmed down version will be sufficient for our purposes.

In [7]:
model_checkpoint = "t5-small"

In [8]:
# load the tokenizer that was used for the t5-small model
tokenizer = tr.AutoTokenizer.from_pretrained(
    model_checkpoint
)  # Use a pre-cached model

 As mentioned above, the IMDB dataset is a binary sentiment dataset. Its labels therefore are encoded as (-1 - unknown; 0 - negative; 1 - positive) values. In order to use this dataset with a text-to-text model like T5, the label set needs to be represented as a string. There are a number of ways to accomplish this. Here, we will simply translate each label id to its corresponding string value.

## Closure Function Example

In [9]:
# closure Function Example Calls
# its use for defining of pre-defined value instead of global variables
def outer_function(x):
    def inner_function(y):
        return x + y  # inner_function has access to x from the outer scope
    
    return inner_function

closure_instance = outer_function(10)  # closure_instance is now a closure with x=10
output_outer_function= closure_instance(10)
output_outer_function

20

In [10]:
def to_tokens(
    tokenizer: tr.models.t5.tokenization_t5_fast.T5TokenizerFast, label_map: dict
) -> callable:
    """
    Given a `tokenizer` this closure will iterate through `x` and return the result of `apply()`.
    This function is mapped to a dataset and returned with ids and attention mask.
    """

    def apply(x) -> tr.tokenization_utils_base.BatchEncoding:
        """From a formatted dataset `x` a batch encoding `token_res` is created."""
        
        target_labels = [label_map[y] for y in x["label"]]
        print('target',target_labels)
        
        token_res = tokenizer(
            x["text"],
            text_target=target_labels,
            return_tensors="pt",
            truncation=True,
            padding=True,
        )
        return token_res

    return apply

In [11]:
imdb_label_lookup = {0: "negative", 1: "positive", -1: "unknown"}


imdb_to_tokens = to_tokens(tokenizer, imdb_label_lookup)
imdb_to_tokens

<function __main__.to_tokens.<locals>.apply(x) -> transformers.tokenization_utils_base.BatchEncoding>

In [25]:
tokenized_dataset = imdb_ds.map(
    imdb_to_tokens, batched=True, remove_columns=["text", "label"]
)

Map:   0%|                                                               | 0/25000 [00:00<?, ? examples/s]

target ['negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negativ

Map:   4%|██                                                | 1000/25000 [00:00<00:15, 1597.66 examples/s]

target ['negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negativ

Map:   8%|████                                              | 2000/25000 [00:01<00:13, 1703.12 examples/s]

...

Map:  96%|███████████████████████████████████████████████  | 48000/50000 [00:29<00:01, 1601.05 examples/s]

target ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'u

Map:  98%|████████████████████████████████████████████████ | 49000/50000 [00:29<00:00, 1596.31 examples/s]

target ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'u

Map: 100%|█████████████████████████████████████████████████| 50000/50000 [00:30<00:00, 1642.74 examples/s]


 ### Step 3 - Setup Training

 The model training process is highly configurable. The [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) class effectively exposes the configurable aspects of the process allowing one to customize them accordingly. Here, we will focus on setting up a training process that performs a single epoch of training with a batch size of 16. We will also leverage `adamw_torch` as the optimizer.

In [26]:
checkpoint_name = "test-trainer"

local_checkpoint_path = os.path.join(local_training_root, checkpoint_name)
local_checkpoint_path

'/tmp/tmpzux8klb4/test-trainer'

In [27]:
# pip install transformers[torch]

In [28]:
training_args = tr.TrainingArguments(
    local_checkpoint_path,
    num_train_epochs=1,  # default number of epochs to train is 3
    per_device_train_batch_size=16,
    optim="adamw_torch",
    report_to=["tensorboard"],
    use_cpu=True
)

 The pre-trained `t5-small` model can be loaded using the [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForSeq2SeqLM) class.

In [29]:
# load the pre-trained model
model = tr.AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, 
)  # Use a pre-cached model

In [24]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

In [30]:
# used to assist the trainer in batching the data
data_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)
trainer = tr.Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

 ### Step 4 - Train

 Before starting the training process, let's turn on Tensorboard. This will allow us to monitor the training process as checkpoint logs are created.

In [31]:
tensorboard_display_dir = f"{local_checkpoint_path}/runs"

In [32]:
 %load_ext tensorboard
 %tensorboard --logdir '{tensorboard_display_dir}'

 Start the fine-tuning process.

In [ ]:
trainer.train()

# save model to the local checkpoint
trainer.save_model()
trainer.save_state()



# persist the fine-tuned model to DBFS
final_model_path = f"{DA.paths.working_dir}/llm04_fine_tuning/{checkpoint_name}"
trainer.save_model(output_dir=final_model_path)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


 ### Step 5 - Predict

In [ ]:
fine_tuned_model = tr.AutoModelForSeq2SeqLM.from_pretrained(final_model_path)



reviews = [
    """
'Despicable Me' is a cute and funny movie, but the plot is predictable and the characters are not very well-developed. Overall, it's a good movie for kids, but adults might find it a bit boring.""",
    """ 'The Batman' is a dark and gritty take on the Caped Crusader, starring Robert Pattinson as Bruce Wayne. The film is a well-made crime thriller with strong performances and visuals, but it may be too slow-paced and violent for some viewers.
""",
    """
The Phantom Menace is a visually stunning film with some great action sequences, but the plot is slow-paced and the dialogue is often wooden. It is a mixed bag that will appeal to some fans of the Star Wars franchise, but may disappoint others.
""",
    """
I'm not sure if The Matrix and the two sequels were meant to have a tigh consistency but I don't think they quite fit together. They seem to have a reasonably solid arc but the features from the first aren't in the second and third as much, instead the second and third focus more on CGI battles and more visuals. I like them but for different reasons, so if I'm supposed to rate the trilogy I'm not sure what to say.
""",
]
inputs = tokenizer(reviews, return_tensors="pt", truncation=True, padding=True)
pred = fine_tuned_model.generate(
    input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]
)



pdf = pd.DataFrame(
    zip(reviews, tokenizer.batch_decode(pred, skip_special_tokens=True)),
    columns=["review", "classification"],
)
display(pdf)

 ## DeepSpeed

 As model architectures evolve and grow, they continually push the limits of available computational resources. For example, some large LLMs having hundreds of billions of parameters making them too large to fit, in some cases, in available GPU memory. Models of this scale therefore need to leverage distributed processing or high-end hardware, and sometimes even both, to support training efforts. This makes large model training a costly undertaking, and therefore accelerating the training process is highly desirable.

 As mentioned above, one such framework that can be leveraged to accelerate the model training process is Microsoft's [DeepSpeed](https://github.com/microsoft/DeepSpeed) [[paper]](https://arxiv.org/pdf/2207.00032.pdf). This framework provides advances in compression, distributed training, mixed precision, gradient accumulation, and checkpointing.

 It is worth noting that DeepSpeed is intended for large models that do not fit into device memory. The `t5-base` model we are using is not a large model, and therefore DeepSpeed is not expected to provide a benefit.

 ### !! Please do not attempt this in Vocareum as it will take more than 5 hours to run and exhaust your compute budget!!




 ### Environment Setup

 The intended use for DeepSpeed is in a distributed compute environment. As such, each node of the environment is assigned a `rank` and `local_rank` in relation to the size of the distributed environment.

 Here, since we are testing with a single node/GPU environment we will set the `world_size` to 1, and both `ranks` to 0.

In [ ]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9994"  # modify if RuntimeError: Address already in use
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

 ### Configuration

 There are a number of [configuration options](https://www.deepspeed.ai/docs/config-json/) that can be set to enhance the training and inference processes. The [ZeRO optimization](https://www.deepspeed.ai/training/#memory-efficiency) settings target reducing the memory footprint allowing for larger models to be efficiently trained on limited resources. 

 The Hugging Face `TrainerArguments` accept the configuration either from a JSON file or a dictionary. Here, we will define the dictionary. 

In [ ]:
zero_config = {
    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {"device": "cpu", "pin_memory": True},
        "allgather_partitions": True,
        "allgather_bucket_size": 5e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 5e8,
        "contiguous_gradients": True,
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto",
            "torch_adam": True,
        },
    },
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto"
}



model_checkpoint = "t5-base"
tokenizer = tr.AutoTokenizer.from_pretrained(
    model_checkpoint, cache_dir=DA.paths.datasets
)

imdb_to_tokens = to_tokens(tokenizer, imdb_label_lookup)
tokenized_dataset = imdb_ds.map(
    imdb_to_tokens, batched=True, remove_columns=["text", "label"]
)

model = tr.AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, cache_dir=DA.paths.datasets
)

 ### Train

In [ ]:
 There are only two changes made to the training setup from above. The first is to set a new checkpoint name. The second is to add the `deepspeed` configuration to the `TrainingArguments`.

 Note: at this time the `deepspeed` argument is considered an experimental feature and may evolve in the future.

In [ ]:
checkpoint_name = "test-trainer-deepspeed"
checkpoint_location = os.path.join(local_training_root, checkpoint_name)
training_args = tr.TrainingArguments(
    checkpoint_location,
    num_train_epochs=3,  # default number of epochs to train is 3
    per_device_train_batch_size=8,
    deepspeed=zero_config,  # add the deepspeed configuration
    report_to=["tensorboard"],
)

data_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)
trainer = tr.Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)



tensorboard_display_dir = f"{checkpoint_location}/runs"



 %load_ext tensorboard
 %tensorboard --logdir '{tensorboard_display_dir}'



trainer.train()

trainer.save_model()
trainer.save_state()



# persist the fine-tuned model to DBFS
final_model_path = f"{DA.paths.working_dir}/llm04_fine_tuning/{checkpoint_name}"
trainer.save_model(output_dir=final_model_path)

 ### Predict

In [ ]:
fine_tuned_model = tr.AutoModelForSeq2SeqLM.from_pretrained(final_model_path)



review = [
    """
           I'm not sure if The Matrix and the two sequels were meant to have a tight consistency but I don't think they quite fit together. They seem to have a reasonably solid arc but the features from the first aren't in the second and third as much, instead the second and third focus more on CGI battles and more visuals. I like them but for different reasons, so if I'm supposed to rate the trilogy I'm not sure what to say."""
]
inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)

pred = fine_tuned_model.generate(
    input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]
)



pdf = pd.DataFrame(
    zip(review, tokenizer.batch_decode(pred, skip_special_tokens=True)),
    columns=["review", "classification"],
)
display(pdf)

 ## Clean up Classroom

 Run the following cell to remove lessons-specific assets created during this lesson.

In [ ]:
tmpdir.cleanup()

 &copy; 2023 Databricks, Inc. All rights reserved.<br/>
 Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
 <br/>
 <a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>